In [3]:
import tushare as ts
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

data_clean = pd.read_csv("future_ss2312_tick.csv")


# 1. 数据加载和预处理
# data_clean = data.sort_values(by='trade_time')
# 确保'close'列是数值型
data_clean['close'] = pd.to_numeric(data_clean['close'], errors='coerce')

# 2. 特征生成
# Example usage:
rolling_windows = {'mean': 300, 'std': 300, 'rsi': 300}
ewm_spans = {'short': 200, 'long': 800, 'signal': 800}


# Calculate rolling mean and standard deviation
data_clean['rolling_mean'] = data_clean['close'].rolling(window=rolling_windows['mean']).mean()
data_clean['rolling_std'] = data_clean['close'].rolling(window=rolling_windows['std']).std()

# Calculate RSI
delta = data_clean['close'].diff()
gain = (delta.where(delta > 0, 0)).fillna(0)
loss = (-delta.where(delta < 0, 0)).fillna(0)
avg_gain = gain.rolling(window=rolling_windows['rsi']).mean()
avg_loss = loss.rolling(window=rolling_windows['rsi']).mean()
rs = avg_gain / avg_loss
data_clean['RSI'] = 100 - (100 / (1 + rs))

# Calculate MACD
short_ema = data_clean['close'].ewm(span=ewm_spans['short'], adjust=False).mean()
long_ema = data_clean['close'].ewm(span=ewm_spans['long'], adjust=False).mean()
data_clean['MACD'] = short_ema - long_ema
data_clean['MACD_signal'] = data_clean['MACD'].ewm(span=ewm_spans['signal'], adjust=False).mean()

# Shift RSI and MACD to use them as features for next timestep
data_clean['RSI_shifted'] = data_clean['RSI'].shift(1)
data_clean['MACD_shifted'] = data_clean['MACD'].shift(1)
data_clean['MACD_signal_shifted'] = data_clean['MACD_signal'].shift(1)

# Define label
data_clean['label'] = (data_clean['close'].shift(-10) > data_clean['close']).astype(int)

# 3. 分割数据
# 分割数据为训练集和测试集
# Convert the 'trade_time' column to datetime
data_clean['trade_time'] = pd.to_datetime(data_clean['trade_time'])

# Now you can filter the data between two dates
train_data = data_clean[(data_clean['trade_time'] >= '2023-09-20 09:00:00') & 
                        (data_clean['trade_time'] < '2023-10-20 09:00:00')]

test_data = data_clean[data_clean['trade_time'] >= '2023-10-20 09:00:00']


features = ['close', 'rolling_mean', 'rolling_std', 'RSI_shifted', 'MACD_shifted', 'MACD_signal_shifted']
X_train = train_data[features]
y_train = train_data['label']
X_train.dropna(inplace=True)
y_train = y_train[X_train.index]

X_test = test_data[features]
y_test = test_data['label']

# 模型训练
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train, y_train)


/var/folders/g3/764mx2lj2gs3qc7l4pz4v2n00000gp/T/ipykernel_46681/402644286.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.dropna(inplace=True)


RandomForestClassifier(n_estimators=200, random_state=42)

In [4]:
def predict_next_move(tick, model, rolling_windows, ewm_spans, historical_data):
    # 将 'last_price' 作为 'current' 进行计算
    tick['current'] = tick['close']
    
    # 将新的 tick 数据追加到历史数据中
    historical_data = pd.concat([historical_data, pd.DataFrame([tick])], ignore_index=True)
    
    
    # 检查我们是否有足够的数据来计算滚动和EWM特征
    if len(historical_data) >= max(rolling_windows['mean'], rolling_windows['std'], rolling_windows['rsi'], ewm_spans['long']):
        # 在历史数据上计算滚动平均和标准差
        historical_data['rolling_mean'] = historical_data['current'].rolling(window=rolling_windows['mean'], min_periods=1).mean()
        historical_data['rolling_std'] = historical_data['current'].rolling(window=rolling_windows['std'], min_periods=1).std()

        # 在历史数据上计算RSI
        delta = historical_data['current'].diff()
        gain = (delta.where(delta > 0, 0)).fillna(0)
        loss = (-delta.where(delta < 0, 0)).fillna(0)
        avg_gain = gain.rolling(window=rolling_windows['rsi'], min_periods=1).mean()
        avg_loss = loss.rolling(window=rolling_windows['rsi'], min_periods=1).mean()
        rs = avg_gain / avg_loss.replace(0, 1)  # 避免除以零
        historical_data['RSI'] = 100 - (100 / (1 + rs))

        # 在历史数据上计算MACD
        short_ema = historical_data['current'].ewm(span=ewm_spans['short'], adjust=False, min_periods=1).mean()
        long_ema = historical_data['current'].ewm(span=ewm_spans['long'], adjust=False, min_periods=1).mean()
        historical_data['MACD'] = short_ema - long_ema
        historical_data['MACD_signal'] = historical_data['MACD'].ewm(span=ewm_spans['signal'], adjust=False, min_periods=1).mean()

        # 将RSI和MACD移位以用作下一个时间步的特征
        historical_data['RSI_shifted'] = historical_data['RSI'].shift(1)
        historical_data['MACD_shifted'] = historical_data['MACD'].shift(1)
        historical_data['MACD_signal_shifted'] = historical_data['MACD_signal'].shift(1)

        # 使用最后一行数据进行预测
        X_new = historical_data.iloc[-1:][['close', 'rolling_mean', 'rolling_std', 'RSI_shifted', 'MACD_shifted', 'MACD_signal_shifted']]
        
        # 检查X_new是否包含NaN值
        if X_new.isnull().values.any():
            # 处理包含NaN值的行（例如，跳过预测或使用占位符值）
            # 例如，我们可以返回None或一个特定的信号表示数据不足
            return None, historical_data
        else:
            # 进行预测
            prediction_proba = model.predict_proba(X_new)
            # 获取预测为类别1的概率
            probability_of_one = prediction_proba[0][1]
            return probability_of_one, historical_data
    else:
        # 数据不足以进行预测
        return None, historical_data




# Initialize historical_data with the correct column names and types if necessary
historical_data = pd.DataFrame()

initial_funds = 100000
funds = initial_funds
stock_quantity = 0
stock_price = 0
buy_threshold = 0.6
sold_threshold = 0.3
transactions = []
minute_count = 0

for idx, row in test_data.iterrows():
    current_probability, historical_data = predict_next_move(row, rf, rolling_windows, ewm_spans, historical_data) 
    # print(current_probability)
    # print(row['trade_time'])
    if current_probability is not None:
        current_price = row['close']
        minute_count = minute_count + 1
            
        if current_probability > buy_threshold and stock_quantity == 0 :
            print(current_probability)
            stock_quantity = funds // current_price
            funds -= stock_quantity * current_price
            fee = stock_quantity * 2
            funds -= fee
            stock_price = current_price
            buy_price = current_price  # 记录买入价格
            print(f"Time: {row['trade_time']} - Action: BUY at {current_price}, Quantity: {stock_quantity},Funds:{funds}")
            transactions.append({
                'action': 'buy',
                'time': row['trade_time'],
                'price': current_price,
                'quantity': stock_quantity,
                'funds_remaining': funds
            })
            minute_count = 0
        elif minute_count > 500 and stock_quantity > 0:
            print(current_probability)
            funds += stock_quantity * current_price
            price_diff = current_price - buy_price  # 计算价格差异
            # 判断价格差异是涨、跌还是平
            if price_diff > 0:
                direction = '涨'
            elif price_diff < 0:
                direction = '跌'
            else:
                direction = '平'
            print(f"Time: {row['trade_time']} - Action: SELL at {current_price}, Quantity: {stock_quantity},Funds:{funds}, Price Change: {direction}")
            transactions.append({
                'action': 'sell',
                'time': row['trade_time'],
                'price': current_price,
                'quantity': stock_quantity,
                'funds_remaining': funds,
                'price_change': direction
            })
            stock_quantity = 0
            buy_price = 0  # 重置买入价格为0
        
    
if len(transactions)>0:
    transactions_df = pd.DataFrame(transactions)
    print(transactions_df['price_change'].value_counts())
transactions_df.to_csv('transactions_tick.csv')

0.65
Time: 2023-10-20 21:00:01 - Action: BUY at 14755.0, Quantity: 6.0,Funds:11458.0
0.535
Time: 2023-10-20 21:04:13 - Action: SELL at 14800.0, Quantity: 6.0,Funds:100258.0, Price Change: 涨
0.605
Time: 2023-10-20 21:05:50 - Action: BUY at 14785.0, Quantity: 6.0,Funds:11536.0
0.23
Time: 2023-10-20 21:10:32 - Action: SELL at 14800.0, Quantity: 6.0,Funds:100336.0, Price Change: 涨
0.63
Time: 2023-10-20 21:52:52 - Action: BUY at 14770.0, Quantity: 6.0,Funds:11704.0
0.195
Time: 2023-10-20 21:57:03 - Action: SELL at 14765.0, Quantity: 6.0,Funds:100294.0, Price Change: 跌
0.645
Time: 2023-10-20 22:21:00 - Action: BUY at 14750.0, Quantity: 6.0,Funds:11782.0
0.14
Time: 2023-10-20 22:25:10 - Action: SELL at 14750.0, Quantity: 6.0,Funds:100282.0, Price Change: 平
0.605
Time: 2023-10-20 22:30:59 - Action: BUY at 14740.0, Quantity: 6.0,Funds:11830.0
0.305
Time: 2023-10-20 22:35:10 - Action: SELL at 14715.0, Quantity: 6.0,Funds:100120.0, Price Change: 跌
0.715
Time: 2023-10-23 00:00:01 - Action: BUY at 

KeyboardInterrupt: 

In [6]:
print(transactions_df['price_change'].value_counts())
print(transactions_df)

涨    17
跌     9
平     4
Name: price_change, dtype: int64
   action                time    price  quantity  funds_remaining price_change
0     buy 2023-10-30 09:35:17  14780.0       6.0          11308.0          NaN
1    sell 2023-10-30 09:39:57  14780.0       6.0          99988.0            平
2     buy 2023-10-30 10:08:23  14745.0       6.0          11506.0          NaN
3    sell 2023-10-30 10:13:12  14750.0       6.0         100006.0            涨
4     buy 2023-10-30 10:33:55  14765.0       6.0          11404.0          NaN
5    sell 2023-10-30 10:39:53  14770.0       6.0         100024.0            涨
6     buy 2023-10-30 11:12:42  14755.0       6.0          11482.0          NaN
7    sell 2023-10-30 11:19:51  14765.0       6.0         100072.0            涨
8     buy 2023-10-30 21:00:01  14625.0       6.0          12310.0          NaN
9    sell 2023-10-30 21:04:12  14645.0       6.0         100180.0            涨
10    buy 2023-10-30 21:25:45  14685.0       6.0          12058.0         